In [1]:
import numpy as np
from functools import cache, lru_cache
from copy import copy, deepcopy

In [2]:
test_input = '''#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########'''

In [3]:
puzzle_input = open('inputs/23').read().strip()

In [4]:
HALL_LEN = 11

In [5]:
LETTERS = 'ABCD'

In [6]:
EMPTY = '.'

In [7]:
LETTER_TO_STACK = {l: i for i, l in enumerate(LETTERS)}

In [8]:
ROOM_LOCATIONS = [2, 4, 6, 8]

In [9]:
ENERGY_AMOUNTS = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [10]:
class State:
    def __init__(self, stacks, hallway, capacity):
        self.stacks = stacks
        self.hallway = hallway
        self.capacity = capacity
        
    def __hash__(self):
        return hash(self.__repr__())
    
    def __repr__(self):
        s = "|"

        for stack in self.stacks:
            for c in stack:
                s += c
            s += "|"

        return f"State object with room capacity {self.capacity}:\n" + "".join(self.hallway) + "\n" + s
    
    # NEED AN EQ METHOD FOR HASHING TO WORK
    def __eq__(self, other):
        return self.__repr__() == other.__repr__()
    
    def is_valid(self):
        return all(self.hallway[i] == EMPTY for i in ROOM_LOCATIONS)
    
    def won(self):
        return all(all(s == l for s in self.stacks[i]) and len(self.stacks[i]) == self.capacity for l, i in LETTER_TO_STACK.items())

In [11]:
@cache
def min_energy_completion(state):
    # enumerate every valid move from this point
    # try each one, recursing
    # return the minimum completion cost from this state

    assert state.is_valid()

    stacks, hallway = state.stacks, state.hallway

    if state.won():
        return 0

    # if there are no valid moves, min is np.inf
    completion_costs = [np.inf]

    ## moving from hall
    # only option is back to your home
    for hall_position, c in enumerate(hallway):
        if c != EMPTY:
            stack_num = LETTER_TO_STACK[c]
            room_location = ROOM_LOCATIONS[stack_num]

            # room could be to our left or our right
            # and in the range we want to exclude ourselves
            if hall_position < room_location:
                left = hall_position + 1
                right = room_location + 1
            elif hall_position > room_location:
                left = room_location
                right = hall_position

            # room not foreign and path not blocked
            if all(s == c for s in stacks[stack_num]) and all(s == '.' for s in hallway[left:right]):
                new_state = deepcopy(state)
                new_state.stacks[stack_num].append(c)
                new_state.hallway[hall_position] = EMPTY

                added_cost = ENERGY_AMOUNTS[c] * (abs(room_location - hall_position) - len(stacks[stack_num]) + state.capacity)
                completion_costs.append(added_cost + min_energy_completion(new_state))

    ## moving from stack
    for stack_num, stack in enumerate(stacks):
        # if empty continue
        if not stack:
            continue

        c = stack[-1]

        # already home and no foreigner underneath us
        # SEE CAVEATS BELOW
        if LETTER_TO_STACK[c] == stack_num and all(d == c for d in stack):
            continue

        room_location = ROOM_LOCATIONS[stack_num]

        left_hallway = hallway[:room_location]
        right_hallway = hallway[room_location+1:]

        # find right-most obstacle in left_hallway
        # find left-most obstacle in right_hallway

        left_obstacle = next((i for i, x in reversed(list(enumerate(left_hallway))) if x != EMPTY), -1)
        right_obstacle = next((i for i, x in enumerate(right_hallway) if x != EMPTY), len(right_hallway))

        # extend left until blocked, extend right until blocked
        valid_positions = list(range(left_obstacle + 1, len(left_hallway))) + list(range(room_location + 1, room_location + 1 + right_obstacle))

        # remove in-front-of-room positions
        valid_positions = set(valid_positions) - set(ROOM_LOCATIONS)

        for p in valid_positions:
            new_state = deepcopy(state)

            new_state.stacks[stack_num].pop()
            new_state.hallway[p] = c

            added_cost = ENERGY_AMOUNTS[c] * (abs(room_location - p) - len(stacks[stack_num]) + state.capacity + 1)
            # print(f"Moving {c} from stack num {stack_num} to hall_position {p} for total cost of {added_cost}")
            completion_costs.append(added_cost + min_energy_completion(new_state))

    # NOTE 
    # HE DIDNT MENTION THE POSSIBILITY THAT AN AMPHIPODS WHO'S HOME COULD LEAVE HOME
    # I WILL ASSUME IT'S BANNED AND/OR NEVER OPTIMAL TO DO
    # BUT IF LATER SOMETHING GOES WRONG CHECK THIS
    # I suppose the condition that once you're in the hall you have to return home is meant to prevent infinite loops.
    # If so, then disallowing amphipods from leaving home makes sense since otherwise it would also allow infinite loops.

    # ALSO, COULD A AMPHIPOD STOP AT THE TOP OF THE STACK, LOGICALLY?
    # I guess not... it will always need to make room for its brethren

    return min(completion_costs)

In [12]:
def parse_p1(puzzle_input):
    stacks = [[], [], [], []]
    
    hallway = list('.' * HALL_LEN)
    
    for line in puzzle_input.splitlines():
        i = 0
        for c in line:
            if c in LETTERS:
                stacks[i].insert(0, c)
                i += 1
                
    return State(stacks, hallway, capacity=2)

def parse_p2(puzzle_input):
    stacks = [[], [], [], []]
    
    hallway = list('.' * HALL_LEN)
    
    for line in puzzle_input.splitlines():
        i = 0
        for c in line:
            if c in LETTERS:
                stacks[i].insert(0, c)
                i += 1

    for i, r in enumerate([['D', 'D'], ['B', 'C'], ['A', 'B'], ['C', 'A']]):
        stacks[i] = [stacks[i][0]] + r + [stacks[i][-1]]

    return State(stacks, hallway, capacity=4)

In [13]:
def p1(puzzle_input):
    state = parse_p1(puzzle_input)
    return min_energy_completion(state)

In [14]:
def p2(puzzle_input):
    state = parse_p2(puzzle_input)
    return min_energy_completion(state)

In [15]:
assert p1(puzzle_input) == 15299
assert p1(test_input) == 12521

In [19]:
assert p2(test_input) == 44169

In [20]:
assert p2(puzzle_input) == 47193